# Practical 5: Spatial operations

So far, the practicals have been introducing you to some of the key concepts of spatial data and some of the basic steps of properly handling things like CRS and measuring distance. This practical will start to shift more to providing you with a set of "tools" and when you might want to use them for analyses. This practical will cover topics in spatial operations that will involve geometric manipulations as well as (and in combination with) non-spatial data operations. 

Rather than dealing with relatively simple datasets as in the earlier practicals, this practical will also introduce a set of real-world datasets that you will be working with for the remainder of the workshop. We will discuss the data in more detail later.

The objectives of this lesson are to:
* understand different overlay operations and spatial relationships
* carry out gemetric manipulations such as buffers, centroids, intersections
* be able to join, select, aggregate, and dissolve GeoDataFrames

In [ ]:
# load
import geopandas as gpd